# 统计门诊数据分析

使用utils中的batch_read函数来读取门诊量数据，需要注意的是，其中有一个`is_filter`字段，用来决定是否使用过滤器对读取的数据进行过滤

我们提取了原始数据中的7个字段：
- 病人ID
- 访问日期
- 性别
- 诊断结果
- 病情描述
- 病历
- 身体检查结果


In [1]:
# -*- coding:utf-8 -*-
from utils import *
from Analyzer import OutpatientVolAnalyzer

topK = 30
# 过滤了复诊取药的数据
data = batch_read(cols=['PATIENT_ID', 'VISIT_DATE', 'SEX', 
                        'DIAG_DESC', 'ILLNESS_DESC', 'MED_HISTORY',
                        'BODY_EXAM'])
# 未过滤复诊取药的数据
no_filter_data = batch_read(cols=['PATIENT_ID', 'VISIT_DATE', 'SEX', 
                        'DIAG_DESC', 'ILLNESS_DESC', 'MED_HISTORY',
                        'BODY_EXAM'], is_filter = False)
data.head()

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,PATIENT_ID,VISIT_DATE,SEX,DIAG_DESC,ILLNESS_DESC,MED_HISTORY,BODY_EXAM
0,97251747,2016/1/1,女,抑郁症,服药1年，病情稳定。,无,无
1,96326999,2016/1/1,女,精神分裂症；抑郁症；双相情感障碍,发呆自笑、自言自语2月。,2月前，无诱因出现自言自语，发呆，自笑，有时手在空中比划，丈夫同事发觉异常，但生活可自理，照...,无
2,98762271,2016/1/1,男,焦虑抑郁状态,情绪低落、紧张、害怕3月。,3月来夜间做噩梦，惊醒后不能再次入睡困难，心慌，心急，心跳快，高兴不起来，自责，担心自己不能...,接触可，表情忧虑，担心，害怕，情绪低落。
3,98545813,2016/1/1,女,抑郁症；躯体化形式障碍,服药后喉咙干涩、烧灼感减轻，仍感嘴里发辣、涩，情绪好转，能干家务。,1年前无明显诱因开始出现心慌心急，心烦，憋闷感，又感到喉咙、嘴唇烧灼；头木；睡眠尚可，目前服...,无
6,98781460,2016/1/1,男,双相情感障碍；精神分裂样精神病；抑郁症,间断言行异常10余年，再发3天。,2002年出现情绪低落，高兴不起来，消极悲观，住院10余天，恢复正常。此后一切如常。2015...,精神检查：意识清，检查合作，存在被害被监控妄想，自知力不存在。


OutpatientVolAnalyzer是一个用于对门诊量数据进行分析的类，我们实现了三种方法对词按照其重要性进行排序的方法，分别是：
- TF-IDF
- Textrank
- Frequency（词出现的频率）

# 病情描述字段的重要词排序（名词）

下面是对病情描述字段进行分析后的结果，可以发现，在病人描述字段中，经过分词以后得到的重要程度比较高的是（三种方法的交集，按照TF-IDF进行排序）：
- 气短
- 不适感
- 家人
- 症状
- 出院
- 服药
- 情绪
- 家属
- 兴趣
- 胸闷
- 言行
- 月余
- 情绪低落
- 躯体
- 母亲
- 头痛
- 患者
- 病史
- 病情
- 心理
- 凭空
- 头晕

In [2]:
analyzer = OutpatientVolAnalyzer(data, focused_fields = ['ILLNESS_DESC'])
ill_des_result = compute_term_rank(analyzer, topK)
display_term_rank_result(ill_des_result)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\01\AppData\Local\Temp\jieba.cache
Loading model cost 0.690 seconds.
Prefix dict has been built succesfully.


Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
         tfidf  textrank    freq
气短    0.146851  0.192053  1324.0
不适感   0.077615  0.110521   595.0
家人    0.061438  0.090328   727.0
症状    0.142107  0.218845  1786.0
出院    0.179123  0.153002  1710.0
服药    0.249690  0.318328  2315.0
情绪    0.467197  0.483920  6392.0
家属    0.311646  0.355565  3360.0
兴趣    0.074557  0.082381   899.0
胸闷    0.159364  0.195679  1441.0
言行    0.188636  0.114836  1742.0
月余    0.410134  0.487119  3257.0
情绪低落  1.180735  0.554723  9125.0
躯体    0.217614  0.258847  2174.0
母亲    0.045059  0.083163   641.0
头痛    0.097841  0.119389  1028.0
患者    0.503209  1.000000  6569.0
病史    0.439888  0.611758  4072.0
病情    0.201490  0.376169  2284.0
心理    0.094710  0.184801  1251.0
凭空    0.073827  0.072894   656.0
头晕    0.085807  0.109815   851.0


# 诊断结果字段的重要词排序（名词）

同样的，我们还可以得到诊断结果字段中，重要的词项，从下面的结果中可以看出主要包括：
- 综合征
- 精神障碍
- 儿童
- 焦虑症
- 状态
- 症状
- 复发性
- 情绪
- 癫痫
- 精神分裂症
- 强迫症
- 性神经症
- 强迫观念
- 形式
- 糖尿病
- 躯体
- 精神病
- 器质性
- 高血压
- 强迫性
- 功能障碍
- 幻觉
- 精神
- 障碍
- 疾病

In [9]:
analyzer = OutpatientVolAnalyzer(data, focused_fields = ['DIAG_DESC'])
diag_des_result = compute_term_rank(analyzer, topK)
display_term_rank_result(diag_des_result)

Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
          tfidf  textrank     freq
综合征    0.022292  0.020055    364.0
精神障碍   0.420465  0.230533   5363.0
儿童     0.055320  0.057884   1096.0
焦虑症    0.802830  0.290863   9669.0
状态     1.383424  1.000000  33359.0
症状     0.121011  0.086483   2474.0
复发性    0.066193  0.020227    713.0
情绪     0.205114  0.200208   4565.0
癫痫     0.042589  0.026928    593.0
精神分裂症  1.182516  0.526521  15182.0
强迫症    0.142783  0.054653   1610.0
性神经症   0.063918  0.020363    701.0
强迫观念   0.054138  0.019033    525.0
形式     0.129343  0.171242   3471.0
糖尿病    0.022560  0.022203    394.0
躯体     0.375731  0.290595   6106.0
精神病    0.318755  0.162680   4826.0
器质性    0.214942  0.118421   2934.0
高血压    0.047666  0.047813    805.0
强迫性    0.096476  0.048120   1074.0
功能障碍   0.036519  0.020970    534.0
幻觉     0.036349  0.023463    554.0
精神     0.057948  0.049799   1390.0
障碍     1.183274  0.879174  23254.0
疾病     0.102525  0.102261   2238.0


# 病历字段的重要词排序（名词）

从病历字段中可以得到的重要的词项主要包括：
- 药物
- 家人
- 诱因
- 家族史
- 症状
- 服药
- 记忆力
- 情绪
- 胸闷
- 心情
- 医院
- 情绪低落
- 食欲
- 体健
- 头痛
- 患者
- 病史
- 病情
- 基本
- 精神
- 感觉

In [4]:
analyzer = OutpatientVolAnalyzer(no_filter_data, focused_fields = ['MED_HISTORY'])
med_history_result = compute_term_rank(analyzer, topK)
display_term_rank_result(med_history_result)

Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
         tfidf  textrank     freq
药物    0.138918  0.670754  13781.0
家人    0.067357  0.413443   6133.0
诱因    0.111612  0.212890   7905.0
家族史   0.092524  0.348913   5805.0
症状    0.235465  0.946940  22771.0
服药    0.300782  0.918988  21458.0
记忆力   0.077513  0.233433   5334.0
情绪    0.190910  0.758243  20098.0
胸闷    0.068960  0.268948   4798.0
心情    0.069152  0.267636   6945.0
医院    0.063866  0.392515   6946.0
情绪低落  0.214206  0.438790  12738.0
食欲    0.110633  0.307567   8043.0
体健    0.096320  0.222092   5072.0
头痛    0.066249  0.282421   5356.0
患者    0.192638  1.000000  19350.0
病史    0.133135  0.436608   9483.0
病情    0.224253  0.748927  19560.0
基本    0.045359  0.359868   6024.0
精神    0.058486  0.348503   6636.0
感觉    0.102950  0.786501  11636.0


# 检查结果字段的重要词排序（名词）

从病历字段中可以得到的重要的词项主要包括：
- 思维
- 症状
- 记忆力
- 情绪
- 情感
- 兴趣
- 表情
- 观念
- 心情
- 交流
- 意志
- 情绪低落
- 体验
- 食欲
- 躯体
- 精神病
- 意识
- 神志
- 消极
- 病情
- 幻觉
- 切题
- 精神

In [15]:
analyzer = OutpatientVolAnalyzer(no_filter_data, focused_fields = ['BODY_EXAM'])
body_exam_result = compute_term_rank(analyzer, topK)
display_term_rank_result(body_exam_result)

Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
         tfidf  textrank     freq
思维    0.151249  0.374130   2647.0
症状    0.289281  0.573241   5627.0
记忆力   0.092042  0.149008   1274.0
情绪    0.142809  0.332677   3024.0
情感    0.156900  0.164037   2701.0
兴趣    0.056746  0.105344   1059.0
表情    0.333122  0.250166   6203.0
观念    0.085227  0.207133   1598.0
心情    0.073314  0.202100   1481.0
交流    0.142190  0.320742   2689.0
意志    0.232739  0.296707   4015.0
情绪低落  0.229662  0.305988   2747.0
体验    0.902767  0.920966  15577.0
食欲    0.066813  0.098695    977.0
躯体    0.143644  0.283633   2221.0
精神病   0.313226  0.600633   4512.0
意识    0.522173  0.619712  10490.0
神志    0.117770  0.122216   1616.0
消极    0.089892  0.212475   1439.0
病情    0.037277  0.090562    654.0
幻觉    0.326182  0.512584   4730.0
切题    0.784199  1.000000   8306.0
精神    0.223204  0.326495   5094.0


# 病情描述字段的重要词排序（形容词）

- 烦躁
- 孤僻
- 疲乏
- 敏感
- 不安
- 悲观
- 乏困
- 急躁
- 郁闷
- 稳定
- 焦虑
- 心慌
- 乏力
- 不适
- 强迫
- 不稳
- 心急
- 明显
- 紧张
- 抑郁
- 心烦

In [11]:
analyzer = OutpatientVolAnalyzer(data, focused_fields = ['ILLNESS_DESC'])
ill_des_result_a = compute_term_rank(analyzer, topK, ['a'])
display_term_rank_result(ill_des_result_a)

Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
       tfidf  textrank     freq
烦躁  0.238723  0.113278   1109.0
孤僻  0.099515  0.031589    373.0
疲乏  0.042455  0.023785    183.0
敏感  0.164363  0.063916    928.0
不安  0.054817  0.061956    305.0
悲观  0.136572  0.067899    685.0
乏困  0.079122  0.024366    224.0
急躁  0.038419  0.029337    163.0
郁闷  0.568188  0.386168   2469.0
稳定  0.314464  0.094098   2394.0
焦虑  0.115965  0.057740    554.0
心慌  0.811038  0.425933   3656.0
乏力  0.130518  0.075718    634.0
不适  0.505339  0.201244   2462.0
强迫  0.067328  0.055900    343.0
不稳  0.284592  0.037503   1352.0
心急  0.976758  0.668021   3933.0
明显  0.044682  0.032089    379.0
紧张  0.469812  0.300166   3051.0
抑郁  0.086245  0.035902    377.0
心烦  2.473103  1.000000  10956.0


# 诊断结果字段的重要词排序（形容词）
- 紊乱
- 不足
- 干燥
- 不安
- 焦虑
- 一般
- 心慌
- 不适
- 消瘦
- 疲劳
- 便秘
- 头疼
- 迟滞
- 疏松
- 强迫
- 躁狂
- 恶劣
- 恐怖
- 惊恐
- 明显
- 抑郁

In [12]:
analyzer = OutpatientVolAnalyzer(data, focused_fields = ['DIAG_DESC'])
diag_des_result_a = compute_term_rank(analyzer, topK, ['a'])
display_term_rank_result(diag_des_result_a)

Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
       tfidf  textrank     freq
紊乱  0.007636  0.008012     51.0
不足  0.000843  0.007399      8.0
干燥  0.001420  0.007479     11.0
不安  0.001435  0.007453     11.0
焦虑  4.053902  1.000000  26680.0
一般  0.000637  0.007550      8.0
心慌  0.003221  0.015266     20.0
不适  0.011770  0.008778     79.0
消瘦  0.001771  0.007507     11.0
疲劳  0.002251  0.011889     16.0
便秘  0.002202  0.009840     14.0
头疼  0.001735  0.007459     11.0
迟滞  0.207135  0.016166   1149.0
疏松  0.013139  0.012840     84.0
强迫  0.330422  0.071088   2319.0
躁狂  0.076735  0.011186    367.0
恶劣  0.017172  0.009737    121.0
恐怖  0.002973  0.007507     21.0
惊恐  0.059013  0.015525    390.0
明显  0.001027  0.007374     12.0
抑郁  3.775662  0.874158  22737.0


# 病历字段的重要词排序（形容词）

- 烦躁
- 疲乏
- 敏感
- 悲观
- 郁闷
- 稳定
- 焦虑
- 一般
- 心慌
- 乏力
- 不适
- 容易
- 心急
- 严重
- 有效
- 明显
- 紧张
- 抑郁
- 心烦

In [13]:
analyzer = OutpatientVolAnalyzer(data, focused_fields = ['MED_HISTORY'])
med_history_result_a = compute_term_rank(analyzer, topK, ['a'])
display_term_rank_result(med_history_result_a)

Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
       tfidf  textrank     freq
烦躁  0.347168  0.295520   5323.0
疲乏  0.127860  0.078245   1819.0
敏感  0.086988  0.051692   1621.0
悲观  0.171195  0.088720   2834.0
郁闷  0.163227  0.104367   2341.0
稳定  0.129146  0.079432   3245.0
焦虑  0.301063  0.241020   4747.0
一般  0.054170  0.049874   1631.0
心慌  0.587444  0.579123   8740.0
乏力  0.183378  0.129098   2940.0
不适  0.308334  0.232927   4958.0
容易  0.097327  0.174481   2335.0
心急  0.869316  1.000000  11553.0
严重  0.073785  0.047857   1799.0
有效  0.086280  0.048229   2116.0
明显  0.387887  0.254990  10859.0
紧张  0.365918  0.370358   7843.0
抑郁  0.244257  0.172487   3524.0
心烦  0.971793  0.927010  14209.0


# 检查结果字段的重要词排序（形容词）

- 平淡
- 稳定
- 清楚
- 心急
- 焦虑
- 一般
- 完整
- 明显
- 紧张
- 心慌
- 强迫
- 不适
- 烦躁
- 适切
- 敏感
- 抑郁
- 悲观
- 心烦

In [16]:
analyzer = OutpatientVolAnalyzer(no_filter_data, focused_fields = ['BODY_EXAM'])
body_exam_result_a = compute_term_rank(analyzer, topK, ['a'])
display_term_rank_result(body_exam_result_a)

Computing TF-IDF...
Computing TextRank...
Computing Frequency...
top 30 intersection are:
       tfidf  textrank     freq
平淡  0.066237  0.038355    496.0
稳定  0.031977  0.036142    391.0
清楚  0.505526  0.121865   5656.0
心急  0.247705  0.209736   1602.0
焦虑  2.215265  1.000000  16998.0
一般  0.151377  0.178830   2218.0
完整  0.047801  0.034267    467.0
明显  0.288175  0.318516   3926.0
紧张  0.148697  0.117838   1551.0
心慌  0.090466  0.068367    655.0
强迫  0.156307  0.087178   1279.0
不适  0.194883  0.094381   1525.0
烦躁  0.137372  0.093642   1025.0
适切  0.439631  0.154147   2104.0
敏感  0.046755  0.023215    424.0
抑郁  2.003986  0.855738  14070.0
悲观  0.126242  0.069163   1017.0
心烦  0.347556  0.221712   2473.0
